In [88]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf

In [89]:
df = pd.read_csv("data_sample1.csv")

In [90]:
#df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1738080 entries, 0 to 1738079
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   transaction           int64  
 1   platform              object 
 2   country_name          object 
 3   journey_id            int64  
 4   channel_id            int64  
 5   timestamp             float64
 6   timestamp_conversion  float64
 7   s                     int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 106.1+ MB


## Transform the transaction column s.t. only  last tp before conversioin has transaction == 1

In [91]:
df['time_diff'] = df['timestamp_conversion'] - df['timestamp'] #create new var for timedifference

In [92]:
df[df['time_diff'] < 0] #All observations where there's a touchpoint after conversion... 34 rows will be ignored

,transaction,platform,country_name,journey_id,channel_id,timestamp,timestamp_conversion,s,time_diff
9564,1,android,Germany,141909,6,5059.231789,5059.130278,1,-0.101511
67236,1,android,Russia,1087921,11,5651.570887,5651.568333,1,-0.002553
94167,1,android,Morocco,1510896,6,4620.550119,4620.509722,1,-0.040397
94534,1,android,Argentina,1514509,6,5353.457879,5353.413056,1,-0.044823
147631,1,android,Israel,2353219,6,3734.558201,3734.493889,1,-0.064312
249822,1,android,Germany,141909,11,5059.282115,5059.130278,1,-0.151837
261735,1,android,South Africa,3831461,11,4882.131187,4882.084167,1,-0.047021
356417,1,ios,Cambodia,2634426,3,1328.453941,1328.450556,1,-0.003386
358682,1,android,Austria,4981254,11,2943.411687,2943.410000,1,-0.001687
462168,1,android,Brazil,6047136,6,5426.905565,5426.612222,1,-0.293343


In [93]:
df.drop(df[df.time_diff < 0].index, inplace = True) #remove these time_diff < 0 i.e. tp after transaction

In [94]:
df = df.sort_values('timestamp')
df = df.sort_values('journey_id')

In [95]:
groups = df.groupby('journey_id').time_diff
min_val = groups.transform(min) #search minimal time_diff in each group <=> closest tp to conversion

cond1 = df.time_diff==min_val #define condition when transaction should be 1

df['transaction'] = np.select([cond1], [1], default = 0) #transform transaction


## Long Journeys

In [96]:
journ_len = df.groupby("journey_id")["s"].count()
print(journ_len.describe())
p1 = 0.99
p2 = 0.9999
print('The ', p1*100, '% quantile is ', np.quantile(journ_len, p1))
print('The ', p2*100, '% quantile is ', np.quantile(journ_len, p2))

count    748466.000000
mean          2.322144
std           5.670744
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max        3143.000000
Name: s, dtype: float64
The  99.0 % quantile is  16.0
The  99.99 % quantile is  100.0


In [97]:
df = df.groupby('journey_id').filter(lambda x: len(x) <= 16)

In [98]:
journ_len = df.groupby("journey_id")["s"].count()
print(journ_len.describe())
p1 = 0.99
p2 = 0.9999
print('The ', p1 * 100, '% quantile is ', np.quantile(journ_len, p1))
print('The ', p2 * 100, '% quantile is ', np.quantile(journ_len, p2))


count    741646.000000
mean          2.041784
std           2.046673
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          16.000000
Name: s, dtype: float64
The  99.0 % quantile is  11.0
The  99.99 % quantile is  16.0


## Remove irrelevant columns

In [99]:
df = df.drop(['s', 'timestamp_conversion', 'time_diff'], axis = 1) #cant be used for prediction

## How to handle object variables

###  Dummy variables for country, platform and channel, better but also huge data

In [100]:
df = pd.get_dummies(df, columns = ['channel_id'], prefix = 'channel', prefix_sep = '_', dtype = float)
df = pd.get_dummies(df, columns = ['country_name'], prefix = 'country', prefix_sep = '_', dtype = float)
df = pd.get_dummies(df, columns = ['platform'], prefix = 'platform', prefix_sep = '_', dtype = float)

### Ordinal Encoder, not really accurate, but doesn't blow up df

In [101]:
#ordinal_encoder = OrdinalEncoder()
#for column in df.columns:
#    if df[column].dtypes == 'object':
#        df[column] = ordinal_encoder.fit_transform(df[[column]])

In [102]:
df.describe()

,transaction,journey_id,timestamp,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,...,country_Vietnam,country_Wallis and Futuna,country_Yemen,country_Zambia,country_Zimbabwe,country_Åland,platform_android,platform_desktop,platform_ios,platform_mobileWeb
count,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,...,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06,1.514281e+06
mean,8.920405e-03,5.827968e+06,4.078644e+03,5.199755e-01,7.659477e-02,1.364377e-01,1.480571e-02,1.311910e-01,3.277001e-02,5.765046e-02,...,6.994078e-03,6.603794e-07,2.047176e-05,8.849084e-05,1.023588e-04,5.216997e-05,1.052843e-02,3.948395e-01,1.496750e-02,5.796645e-01
std,9.402573e-02,4.145757e+06,1.429707e+03,4.996010e-01,2.659475e-01,3.432529e-01,1.207746e-01,3.376092e-01,1.780341e-01,2.330814e-01,...,8.333766e-02,8.126373e-04,4.524528e-03,9.406544e-03,1.011674e-02,7.222692e-03,1.020666e-01,4.888164e-01,1.214227e-01,4.936129e-01
min,0.000000e+00,7.000000e+00,9.965556e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.257795e+06,2.974278e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,4.995115e+06,4.142417e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,0.000000e+00,8.980681e+06,5.272426e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.497620e+07,6.385067e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Next step: transform to tensor